Calculate and compare RMSD values for all CDRH3s in reduction clusters and SPACE2 structural configuration clusters 

## Imports and setups

Substitute variables with appropriate file paths/selections

In [ ]:
import sys
sys.path.append('/Users/isaacdaviet/Desktop/thesis/python_versions')
# replace with directory containing the .py calculation files below
import SPACE2_analysis as sp2
import pdb_extraction as pdb
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display, clear_output
import time
from Bio.SVDSuperimposer import SVDSuperimposer
from Bio.PDB import PDBParser
import RMSD_calcs as rmsd

# python_files_folder = '/Users/isaacdaviet/Desktop/thesis/python_versions' # replace with directory containing the .py calculation files below
# import sys
# sys.path.append(python_files_folder)
# import importlib
# import RMSD_calcs as rmsd

# # Reload the module
# importlib.reload(rmsd)

## Calculate Average RMSD for dataframe containing entire dataset + ISEQ column

Can filter for binder/non binder or any other column included in the dataframe, as long as the iseq column is there. Generates updating graph tracking how the RMSD averages change over time. Not essential, can skip to next section to generate simplified distribution graph

In [ ]:
### Input Folders

igfold_folder = '/Users/isaacdaviet/Desktop/mason_igfold_models/mason_igfold_models/igfold_outfile'
# Folder containing ALL PDB files analyzed. 

iseq_incl_file = '/Users/isaacdaviet/Desktop/results/SPACE2_analysis/PCA_manual_clusters/SPACE2_cluster_replotting/mason_pca-space2_superclusters.csv'
# Cluster file containing individual iseq #s to be analyzed. Exampled given uses PDB files containing only positions in CDRH3

igfold_pdb_file_name_format = '/Users/isaacdaviet/Desktop/mason_igfold_models/mason_igfold_models/igfold_CDRH3_pdbs/mHER_H3_LABEL_unique_fv_ISEQ_igfoldCDRH3_ONLY.pdb'
# File path format using 'ISEQ' as string to be replaced by given ISEQ # 

save_folder = '/Users/isaacdaviet/Desktop/results/SPACE2_analysis/Mason_RMSD_stats'

project = 'Mason'


### Dataframe Filters
#### first level filter to remove any inconvenient cells
prefilter_column = 'n_PCs'
contains = False
prefilter_value = None

#### Second level filter, used to specify which label to analyze
column = 'Labels' # Set to None if wish to test all seqs or 'Labels' to analyze binders or non-binders only
column_filter = 'Non Binder' # Set to 'Binder' or 'Non Binder' to EXCLUDE a specific label 

### Number of sequences to sample (recommend setting pc_df to 1 and using # of binder sequences in dataet for both binders and non-binder)
n_seq = 8955 # exact number of randomized CDRH3's to use (default in function is 'all' to analyze all sequences)
pc_df= 1 # percentage of resulting df to use (default in function is 1 or 100%)

n_test = 3 # number of repeats to conduct




iseq_df = pd.read_csv(iseq_incl_file)
if prefilter_column is not None:
    iseq_df = iseq_df[iseq_df[prefilter_column] == prefilter_value] if contains == True else iseq_df[iseq_df[prefilter_column] != prefilter_value] #pre filter data
    cont_ext = 'is' if contains == True else 'isnot'
    ### Pre filters to explore: specific UMAP reductions/clusters, maybe look at the best graph for each metric. Also look at each metric separately?

all_rmsd_calcs, all_rmsd_means, all_rmsd_medians = [],[],[]

for i in range (1, n_test+1):


    save_final_graph_file = os.path.join(save_folder, f'{project}_RMSD-stats_{column_filter}_{column}_{pc_df*100}PC-of-{n_seq}Seqs-{prefilter_column}{cont_ext}{prefilter_value}_test{i}.png') if save_folder is not None else None

    rmsd_calcs, rmsd_means, rmsd_medians, n_sequences = rmsd.calculate_avg_rmsd_of_dataset(iseq_df, column, column_filter, igfold_pdb_file_name_format, title, n_sequences = n_seq, pc_selected_seqs = pc_df, save_final_graph_file = save_final_graph_file, show_updating_graph = 'n')

    all_rmsd_calcs.extend(rmsd_calcs)
    all_rmsd_means.extend(rmsd_means)
    all_rmsd_medians.extend(rmsd_medians)

    all_max_rmsd = round(max(all_rmsd_calcs), 4)
    all_min_rmsd = round(min(all_rmsd_calcs), 4)
    final_rmsd_avg = round(np.mean(all_rmsd_means), 4)
    final_rmsd_median = round(np.median(all_rmsd_medians), 4)
    pc_mean_to_median = round((min([final_rmsd_avg, final_rmsd_median]) / max([final_rmsd_avg, final_rmsd_median])) * 100, 2)


string_to_save = f'RMSD analysis of {project} {column_filter} {column} at {pc_df*100}% of {n_seq} Seqs\nPrefilter: {prefilter_column}{cont_ext}{prefilter_value}\n\tMaximum RMSD: {all_max_rmsd}\n\tMinimum RMSD: {all_min_rmsd} \n\tAverage RMSD: {final_rmsd_avg}\n\tMedian RMSD: {final_rmsd_median}\n\tMeans/Median Ratio: {pc_mean_to_median}%'

print(string_to_save)


if save_folder is not None:

    string_file = os.path.join(save_folder, f'SUMMARY-{project}_RMSD-stats_{column}_{column_filter}_{pc_df*100}PC-of-{n_seq}Seqs-PreFilt{prefilter_column}{cont_ext}{prefilter_value}.txt')

    with open(string_file, 'w') as file:
        file.write(string_to_save)

## Isolate the CDRH3 structures from full IgG PDB files, if necessary

In [ ]:
pdb_folder = r'/Users/isaacdaviet/Desktop/mason_igfold_models/mason_igfold_models/igfold_outfile'
# Folder containing original full IgG PDB files
output_folder = r'/Users/isaacdaviet/Desktop/mason_igfold_models/mason_igfold_models/igfold_CDRH3_pdbs'

rmsd.add_all_cdrh3_pdb_files(pdb_folder, output_folder)

# RMSD Probabability distribution graphs.

Generate probability distribution graphs for simplified view of RMSD ranges contained in dataset.
First section calculates RMSD values and displays values for each test conducted (n_test variable), second section generates the graph to allow for editing. Final plot displays means/medians/ranges for each curve.

In [ ]:

igfold_folder = '/Users/isaacdaviet/Desktop/mason_igfold_models/mason_igfold_models/igfold_outfile'
# Folder Containing PDB files

iseq_incl_file = '/Users/isaacdaviet/Desktop/results/SPACE2_analysis/UMAP_reductions/SPACE2_cluster_replotting/mason_umap-space2_superclusters.csv'
# File containing 'iseq' column for use as replacement input

igfold_pdb_file_name_format = '/Users/isaacdaviet/Desktop/mason_igfold_models/mason_igfold_models/igfold_CDRH3_pdbs/mHER_H3_LABEL_unique_fv_ISEQ_igfoldCDRH3_ONLY.pdb'
# File path & name format using 'ISEQ' as string to be replaced by given ISEQ # 

save_folder = '/Users/isaacdaviet/Desktop/results/SPACE2_analysis/Mason_RMSD_stats'
# save_folder = None

project = 'Mason'

### RMSD Calculation Parameters
n_seq = 8955 # exact number of randomized CDRH3's to use (default in function is 'all' to analyze all sequences, recommend setting to total # of binders for all analysis at pc_df = 1)
pc_df= 1 # percentage of resulting df to use (default in function is 1 or 100%)
n_test = 3 # number of repeats to conduct (recommend 3 to confirm results)

exclude_full_library_curve = False # Set to True to exclude full library (both binders & non binders) RMSD analysis




### RMSD Calculations
iseq_df = pd.read_csv(iseq_incl_file)

# title = f'{project} RMSD Evolution - {pc_df*100}% of {n_seq} {column} Sequences Where {prefilter_column}{cont_ext}{prefilter_value}'


b_rmsd_calcs =  []
nb_rmsd_calcs = []
fl_rmsd_calcs = []

for i in range (1, n_test+1):


    b_rmsds= rmsd.calculate_rmsd_averages(iseq_df, 'Labels', 'Binder', igfold_pdb_file_name_format, n_sequences = n_seq)

    nb_rmsds= rmsd.calculate_rmsd_averages(iseq_df, 'Labels', 'Non Binder', igfold_pdb_file_name_format, n_sequences = n_seq)

    b_rmsd_calcs.extend(b_rmsds)
    nb_rmsd_calcs.extend(nb_rmsds)

    if exclude_full_library_curve is not True:
        fl_rmsds = rmsd.calculate_rmsd_averages(iseq_df, 'Labels', 'all', igfold_pdb_file_name_format, n_sequences = n_seq)
        fl_rmsd_calcs.extend(fl_rmsds)



In [ ]:
exclude_full_library_curve = True # True = exclude full libary curve from final graph



### Graph Generation
b_max_rmsd = round(max(b_rmsd_calcs), 2)
b_min_rmsd = round(min(b_rmsd_calcs), 2)

nb_max_rmsd = round(max(nb_rmsd_calcs), 2)
nb_min_rmsd = round(min(nb_rmsd_calcs), 2)

if exclude_full_library_curve is not True:
    fl_max_rmsd = round(max(fl_rmsd_calcs), 2)
    fl_min_rmsd = round(min(fl_rmsd_calcs), 2)

sns.set(style= 'whitegrid')
fig, ax = plt.subplots(figsize=(10,6))

sns.kdeplot(nb_rmsd_calcs, ax =ax, label = f'Non Binders: {nb_min_rmsd}-{nb_max_rmsd}', color = 'blue')

sns.kdeplot(b_rmsd_calcs, ax =ax, label = f'Binder: {b_min_rmsd}-{b_max_rmsd}', color = 'red')

sns.kdeplot(fl_rmsd_calcs, ax =ax, label = f'Full Library: {fl_min_rmsd}-{fl_max_rmsd}', color = 'green') if exclude_full_library_curve is not True else None


if exclude_full_library_curve is not True:
    for data, color, label in zip([b_rmsd_calcs, nb_rmsd_calcs, fl_rmsd_calcs], ['red', 'blue', 'green'], ['Binder', 'Non Binder', 'All']):
        ax.axvline(round(np.mean(data), 2), color=color, linestyle='--', label=f'{label.capitalize()} Avg: {np.mean(data):.4f}')
        ax.axvline(round(np.median(data), 2), color=color, linestyle='-', label=f'{label.capitalize()} Median: {np.median(data):.4f}')

else:
    for data, color, label in zip([b_rmsd_calcs, nb_rmsd_calcs], ['red', 'blue'], ['Binder', 'Non Binder']):

        ax.axvline(round(np.mean(data), 2), color=color, linestyle='--', label=f'{label.capitalize()} Avg: {np.mean(data):.4f}')
        ax.axvline(round(np.median(data), 2), color=color, linestyle='-', label=f'{label.capitalize()} Median: {np.median(data):.4f}')

# Customize plot
ax.set_xlabel('RMSD')
ax.set_ylabel('Density')
ax.set_title('RMSD Distribution By Label')
ax.legend()

# Show plot
plt.show()

## UMAP multicluster RMSD calculation

Calculate the RMSD of the SPACE2 clusters contained within a single UMAP reduction cluster, when applicable

#### Generate RMSD Summary Sheet

In [ ]:
igfold_folder = '/Users/isaacdaviet/Desktop/mason_igfold_models/mason_igfold_models/igfold_CDRH3_pdbs'
# Folde containing PDB files to be analyzed

all_summaries_xl = '/Users/isaacdaviet/Desktop/results/SPACE2_analysis/PCA_manual_clusters/Redo/pca_reduced_all_summaries.xlsx'
# all_summaries file generated in previous sections

reduction_type = 'PCA'

rmsd.rmsds_summary_sheet(all_summaries_xl, igfold_folder, reduction_type)

### Violin Plots of RMSD values

Generates violin plots of calculated UMAP reduction RMSD values, generates the following plots:

UMAP:

    - x, y, labels_filter = 'n_Abs%', 'label', 'all'
    - x, y, labels_filter = 'n_Abs%', 'metric', 'all'
    - x, y, labels_filter = 'n_Abs%', 'label', 'Binder'
    - x, y, labels_filter = 'n_Abs%', 'label', 'Non Binder'

    - x, y, labels_filter = 'avg_rmsd', 'label', 'all'
    - x, y, labels_filter = 'avg_rmsd', 'metric', 'all'
    - x, y, labels_filter = 'avg_rmsd', 'label', 'Binder'
    - x, y, labels_filter = 'avg_rmsd', 'label', 'Non Binder'

    - x, y, labels_filter = '1_vs_2', 'label', 'all'
    - x, y, labels_filter = '1_vs_2', 'metric', 'all'
    - x, y, labels_filter = '1_vs_2', 'label', 'Binder'
    - x, y, labels_filter = '1_vs_2', 'label', 'Non Binder'

PCA:

    - x, y, labels_filter = 'n_Abs%', 'label', 'all'
    - x, y, labels_filter = '1_vs_2', 'label', 'all'
    - x, y, labels_filter = 'avg_rmsd', 'label', 'all'
    - x, y, labels_filter = 'avg_rmsd', 'n_SPACE2_clusters', 'all'
    - x, y, labels_filter = 'avg_rmsd', 'n_SPACE2_clusters', 'Binder'
    - x, y, labels_filter = 'avg_rmsd', 'n_SPACE2_clusters', 'Non Binder' 

NOTE: contains commented code to generate violin plots based on n_abs% following similar format as RMSD violin plots that can be uncommented if necessary o plots not generated in previous sections   

####

In [ ]:
### Files to import
umap_all_summaries_file ='/Users/isaacdaviet/Desktop/results/SPACE2_analysis/UMAP_reductions/structural_clusters/umap_all_summaries.xlsx'
umap_save_path = '/Users/isaacdaviet/Desktop/results/SPACE2_analysis/UMAP_reductions/structural_clusters/all_summaries graphs'

pca_all_summaries_file = '/Users/isaacdaviet/Desktop/results/SPACE2_analysis/PCA_manual_clusters/Redo/pca_reduced_all_summaries.xlsx'
pca_save_path = '/Users/isaacdaviet/Desktop/results/SPACE2_analysis/PCA_manual_clusters/Redo'

reduction_type = 'PCA' # set to 'all', 'UMAP', or 'PCA' depending on which reduction type to analyze

### Recommended plots to generate:
#### format: [sheet in all_summaries file, reduction_type, labels_filter, x_axes, y_axes]
#### After adding/removing any violin plots, be sure to update rmsd_dict dictionary

##### UMAP Plots
a = ['multi_cluster_rmsd', 'UMAP', 'all', 'label', 'avg_rmsd']
b = ['multi_cluster_rmsd', 'UMAP', 'all', 'label', '1_vs_2']
c = ['multi_cluster_rmsd', 'UMAP', 'all', 'metric', 'avg_rmsd']
d = ['multi_cluster_rmsd', 'UMAP', 'Binder', 'metric', 'avg_rmsd']
e = ['multi_cluster_rmsd', 'UMAP', 'Non Binder', 'metric', 'avg_rmsd']
f =['multi_cluster_rmsd', 'UMAP', 'all', 'metric', '1_vs_2']
g =['multi_cluster_rmsd', 'UMAP', 'Binder', 'metric', '1_vs_2']
h =['multi_cluster_rmsd', 'UMAP', 'Non Binder', 'metric', '1_vs_2']

##### PCA Plots
i =['multi_cluster_rmsd', 'PCA', 'all', 'label', 'avg_rmsd']
j =['multi_cluster_rmsd', 'PCA', 'all', 'n_SPACE2_clusters', 'avg_rmsd']
k =['multi_cluster_rmsd', 'PCA', 'all', 'label', '1_vs_2']
l =['multi_cluster_rmsd', 'PCA', 'Binder', 'n_SPACE2_clusters', 'avg_rmsd']
m =['multi_cluster_rmsd', 'PCA', 'Non Binder', 'n_SPACE2_clusters', 'avg_rmsd']

rmsd_dict = {'UMAP': [a,b,c,d,e,f,g,h], 'PCA': [i,j,k,l,m]} # Update if adding/removing plots to generate






### PLot generation
inner_plot_format = 'box'
title_size = 9
project_name = 'Mason'

for key, items in rmsd_dict.items():
    if key == reduction_type or reduction_type == 'all':
        all_summaries_file = umap_all_summaries_file if key == 'UMAP' else pca_all_summaries_file

        input_df = pd.read_excel(all_summaries_file, sheet_name='multi_cluster_rmsds')
        save_path = umap_save_path if key == 'UMAP' else pca_save_path

        for sheet, reduction_type, labels_filter, x, y in items:
            ext = f'{labels_filter}s only' if labels_filter != 'all' else 'all points'

            averages = True if y == 'avg_rmsd' else False

            plt_title = f'Average RMSD of {reduction_type} Clusters Containing Multiple SPACE2 Clusters\n{x} - {ext}' if y == True else f'RMSD of {y[0]} and {y[-1]} Largest SPACE2 Clusters Contained in Single {reduction_type} Cluster \n {x} - {ext}'

            file_name = f'{project_name}_{reduction_type}_RMSD-{y}_{x}_{labels_filter}.png'

            data = input_df[input_df['label'] == labels_filter] if labels_filter != 'all' else input_df
            labels_filter

            if reduction_type != 'PCA':
                data['metric'] = data['reduction_file'].apply(rmsd.get_metric)

            rmsd.summaries_violin_plot(x, y, data, plt_title, file_name, title_size, save_path, inner_plot=inner_plot_format)



##### Additional violin plots based on n_abs% values, not necessary if already done in previous sections. 
# n = ['all_summaries', 'UMAP', 'all', 'label', 'n_abs%']
# o = ['all_summaries', 'UMAP', 'all', 'metric', 'n_abs%']
# p = ['all_summaries', 'UMAP', 'Binder', 'metric', 'n_abs%']
# q = ['all_summaries', 'UMAP', 'Non Binder', 'metric', 'n_abs%']

# r = ['all_summaries', 'PCA', 'all', 'label', 'n_abs%']
# s = ['all_summaries', 'PCA', 'all', 'component_1', 'n_abs%']
# t = ['all_summaries', 'PCA', 'Binder', 'component_1', 'n_abs%']
# u = ['all_summaries', 'PCA', 'Non Binder', 'component_1', 'n_abs%']
# v = ['all_summaries', 'PCA', 'all', 'component_2', 'n_abs%']
# w = ['all_summaries', 'PCA', 'Binder', 'component_2', 'n_abs%']
# x = ['all_summaries', 'PCA', 'Non Binder', 'component_2', 'n_abs%']

# n_abs_dict = {'UMAP': [n, o, p, q], 'PCA': [r, s, t, u, v, w, x]}

# for key, items in n_abs_dict.items():
#     if key == reduction_type or reduction_type == 'all':
#         all_summaries_file = umap_all_summaries_file if key == 'UMAP' else pca_all_summaries_file

#         input_df = pd.read_excel(all_summaries_file, sheet_name='all_summaries')
#         save_path = umap_save_path if key == 'UMAP' else pca_save_path

#         for sheet, reduction_type, labels_filter, x, y in items:

#             ext = f'{labels_filter}s only' if labels_filter != 'all' else 'all points'

#             plt_title = f'Percentages of {reduction_type} Cluster Contained Within Associated SPACE2 Clusters Separated\n{x}' if labels_filter == 'all' else f'Percentages of {reduction_type} Cluster Contained Within Associated SPACE2 Clusters Separated\n{x} - {ext}'

#             file_name = f'{project_name}_{reduction_type}_nAbsPC-{y}_{x}_{labels_filter}.png'

#             data = input_df[input_df['label'] == labels_filter] if labels_filter != 'all' else input_df
#             labels_filter

#             if reduction_type != 'PCA':
#                 data['metric'] = data['reduction_file'].apply(rmsd.get_metric)

#             rmsd.summaries_violin_plot(x, y, data, plt_title, file_name, title_size, save_path, inner_plot=inner_plot_format)
